# Sock Market Screener

1) pré-sélection de la liste d'actions

2) test de la strat sur la liste d'actions

3) Affiner la sélection d'actions

4) Afficher les alertes en temps réel sur les actions

In [1]:
# Imports
from tradingview_screener import Query, col
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta  # Technical Analysis Indicators
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
import requests
from io import StringIO

# 1) pré-sélection de la liste d'actions

Critères de sélection des actions :
- marché Nasdaq et NYSE
- price 25 to 250 USD
- market cap supérieur ou égal à 1B USD
- Avg Volume 30D supérieur à 1M
- Volume supérieur à 1M
- Change supérieur à 0%
- Rel Vol at Time supérieur à 1.2
- SMA(50) inférieur au prix
- SMA(100) inférieur au prix
- SAM(200) inférieur au prix
- VWAP 5m inférieur ou égal au prix
- EMA(8) 5m inférieur ou égal au prix

In [12]:
#TradingView screener

# Requête choix des actions long
tickers = (
    Query()
    .select('name', 'close', 'volume', 'relative_volume_10d_calc')
    .where(
        col('type') == 'stock',
        col('exchange').isin(['NASDAQ', 'NYSE']),
        col('close').between(25, 250),
        col('market_cap_basic') >= 1_000_000_000,
        col('average_volume_30d_calc') > 1_000_000,
        col('volume') > 1_000_000,
        col('change') > 0,
        col('relative_volume') > 1.2,
        col('SMA50') < col('close'),
        col('SMA100') < col('close'),
        col('SMA200') < col('close'),
        #les 2 dernières conditions ne sont pas dispos en 5m avec l'API
    )
    .order_by('volume', ascending=False)
    .limit(3000)
    .get_scanner_data()
)

# Affichage
#print(tickers)
#tickers est au format tuple

# Extraction du DataFrame depuis le tuple
df = tickers[1]

# Extraction de la colonne 'name' sous forme de liste
tickers_list = df['name'].tolist()

# Affichage
print(tickers_list)

print("Il y a",len(tickers_list), "actions extraites" )


['AMD', 'RF', 'CCL', 'CSX', 'TEM', 'BE', 'NKE', 'USB', 'ENB', 'LITE', 'TFC', 'CRSP', 'TPR', 'LMND', 'PGY', 'LEN', 'RPRX', 'LQDA', 'DHI', 'RTX', 'Z', 'SMMT', 'BCE', 'BX', 'OUST', 'LVS', 'AS', 'INSM', 'PFGC', 'FL', 'HGV', 'FYBR', 'SKX', 'NXPI', 'DG', 'BLDR', 'EL', 'DLTR', 'HAS', 'MTG', 'INCY', 'COLB', 'HOG', 'PHM', 'IONS', 'ALLY', 'REZI', 'AMTM', 'JEF', 'AAP', 'ALK', 'CNM', 'ST', 'PSTG', 'TD', 'RY', 'WSM', 'ASO', 'TOL', 'PII', 'STT', 'SLM', 'TMHC', 'TXT', 'NEGG', 'PARR', 'XYL', 'KBH', 'BNS', 'RBA', 'PNW', 'MTH', 'FERG']
Il y a 73 actions extraites


In [8]:
#Rajout des 2 dernières conditions avec yahoo finance

results = []

for symbol in tickers_list:
    try:
        ticker = yf.Ticker(symbol)
        info = ticker.info
        price = info.get('currentPrice')
        vol = info.get('volume', 0)

        # Données intraday 5m pour VWAP et EMA8
        intraday = ticker.history(period='1d', interval='5m')
        if len(intraday) < 8: continue

        vwap = (intraday['Close'] * intraday['Volume']).cumsum() / intraday['Volume'].cumsum()
        vwap_last = vwap.iloc[-1]
        ema8 = intraday['Close'].ewm(span=8, adjust=False).mean().iloc[-1]

        if not (vwap_last <= price and ema8 <= price): continue

        results.append({
            'ticker': symbol,
        })

    except Exception:
        continue

df = pd.DataFrame(results)
print(df)

print("Il y a",len(df), "actions sélectionnées pour tester la stratégie" )


  ticker
0    AMD
1     RF
2   OUST
3    HGV
4   INCY
5   AMTM
6    WSM
Il y a 7 actions sélectionnées pour tester la stratégie


In [13]:
#Rajout de la vwap sans ema8

results = []

for symbol in tickers_list:
    try:
        ticker = yf.Ticker(symbol)
        info = ticker.info
        price = info.get('currentPrice')
        vol = info.get('volume', 0)

        # Données intraday 5m pour VWAP et EMA8
        intraday = ticker.history(period='1d', interval='5m')

        vwap = (intraday['Close'] * intraday['Volume']).cumsum() / intraday['Volume'].cumsum()
        vwap_last = vwap.iloc[-1]

        if not (vwap_last <= price): continue
        results.append({
            'ticker': symbol,
        })

    except Exception:
        continue

df = pd.DataFrame(results)
print(df)

print("Il y a",len(df), "actions sélectionnées pour tester la stratégie" )


   ticker
0     AMD
1      RF
2     CCL
3     CSX
4     NKE
..    ...
61    BNS
62    RBA
63    PNW
64    MTH
65   FERG

[66 rows x 1 columns]
Il y a 66 actions sélectionnées pour tester la stratégie


# 2) test de la strat sur la liste d'actions